#### This is to analyze intensity colocalization by plotting line-scane gray value of different channels 

In [1]:
import os
import re
from skimage import io, restoration
import pandas as pd
import numpy as np
from scipy.ndimage import gaussian_filter
from skimage.draw import line
import matplotlib.pyplot as plt

##### data image

In [ ]:
cell_dir = r"where the cell images are stored"
raw_image_name = ".tif"

####
* read coordinates from txt file and load image

In [ ]:
# read txt file
for item in os.listdir(cell_dir):
    if re.match(".*.txt", item):
        coordinates = pd.read_csv(os.path.join(cell_dir,item), sep="\t",header=None)
    if re.match(raw_image_name, item):
        raw = io.imread(os.path.join(cell_dir,item)).astype(np.float32)
        bg_substracted = np.stack([raw[...,i] - restoration.rolling_ball(raw[...,i],radius=100) for i in range(raw.shape[-1])],axis=-1)
        raw_smoothed=np.stack([gaussian_filter(bg_substracted[...,i],sigma=1,mode="nearest",truncate=3) for i in range(raw.shape[-1])],axis=-1)
        # normalize the image by min-max
        raw_smoothed = np.stack(
            [(raw_smoothed[...,i] - raw_smoothed[...,i].min()) / (raw_smoothed[...,i].max() - raw_smoothed[...,i].min()) for i in range(raw.shape[-1])],axis=-1
            )

In [ ]:
# find the location of the cell whose value matches a expression in the dataframe
def find_cell(coordinates, expression):
    for index, row in coordinates.iterrows():
        for item in row:
            if re.match(expression, item):
                # find the index of a character in the string
                item = item[item.index("(")+1:item.index(")")]
                # split the string into a list
                coordinate = list(map(int, item.split(",")))
                return coordinate

In [ ]:
# this is extract the coordinates of two ends of the line
x1, y1, x2, y2 = find_cell(coordinates=coordinates, expression=".*makeLine.*")
print(x1, y1, x2, y2)

# Generate pixel coordinates along the line
rr, cc = line(y1, x1, y2, x2)

# find the gray value of the line
red_value = raw_smoothed[rr,cc,0]
green_value = raw_smoothed[rr,cc,1]
blue_value = raw_smoothed[rr,cc,2]

# scale line length in micron
distance = np.arange(0, len(red_value), 1) * 0.08
min_x = 0
max_x = int(distance.max())

# find the index of distance where the value is greater than 1 micron, and less than 8 micron
index = np.where((distance > min_x) & (distance < max_x))
# index = np.where(distance < 7)
new_distance = distance[index]
new_red_value = red_value[index]
new_green_value = green_value[index]
new_blue_value = blue_value[index]

# plot gray value of each channel along the line
# plot line scanning
# set figure size based on the distance and y-axis range
x = new_distance.max()
y = 1.2*3

fig, ax = plt.subplots(1,1,figsize=(x,y),dpi=300)
rpoa2, = ax.plot(new_distance, new_green_value,"g")
dao5, = ax.plot(new_distance, new_red_value,"r")
lpd7, =ax.plot(new_distance, new_blue_value,"b")

# change x-axis ticks
ax.set_xticks(np.arange(min_x, max_x,1))
ax.set_yticks(np.arange(0, 1.2,1))

# hide x and y axis labels
ax.set_yticklabels([])

# turn off top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.tight_layout()

plt.savefig("%s/%s.svg" %(cell_dir, "plot_smooth"), facecolor='w', dpi=300,transparent=True)